In [1]:
import os
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

folder = 'fig'

In [4]:
src = cv2.imread(Path(folder, 'checkerboard.png'))

if src is None:
    print('image read failed')
    sys.exit()

src_lists = [[218, 49], [691, 47], [832, 516], [68, 527]]
h, w = src.shape[:2]

srcPoints = np.array(src_lists, np.float32)
dstPoints = np.array([[0, 0], [w-1, 0], [w-1, h-1], [0, h-1]], np.float32)

pers_mat = cv2.getPerspectiveTransform(srcPoints, dstPoints)
print(pers_mat)
dst = cv2.warpPerspective(src, pers_mat, (0, 0))

cv2.namedWindow('src', cv2.WINDOW_AUTOSIZE)
cv2.imshow('src', src)
cv2.imshow('dst', dst)

while True:
    if cv2.waitKey() == ord('q'):
        break

cv2.destroyAllWindows()

[[ 2.13761247e+00  6.70798891e-01 -4.98868663e+02]
 [ 8.22754210e-03  1.94581371e+00 -9.71384758e+01]
 [-1.69199517e-05  1.37469054e-03  1.00000000e+00]]


In [8]:
img = cv2.imread(Path(folder, 'plates.png'), cv2.IMREAD_GRAYSCALE)

sobel_x = np.array([[-1, 0, 1],
                    [-2, 0, 2],
                    [-1, 0, 1]], np.float32)

dx = cv2.filter2D(img, -1, sobel_x)
dy = cv2.filter2D(img, -1, sobel_x.T)

cv2.imshow('img', img)
cv2.imshow('dx', dx)
cv2.imshow('dy', dy)

cv2.waitKey()
cv2.destroyAllWindows()

In [10]:
img = cv2.imread(Path(folder, 'plates.png'), cv2.IMREAD_GRAYSCALE)

dx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
dy = cv2.Sobel(img, cv2.CV_32F, 0, 1)

mag = cv2.magnitude(dx, dy) # float32
mag = np.clip(mag, 0, 255).astype(np.uint8)
ret, dst = cv2.threshold(mag, 100, 255, cv2.THRESH_BINARY)

cv2.imshow('img', img)
cv2.imshow('mag', mag)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
img = cv2.imread(Path(folder, 'checkerboard.png'))
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

edge = cv2.Canny(img_gray, 100, 200)

lines = cv2.HoughLinesP(edge, 1, np.pi/360, 80, minLineLength=20, maxLineGap=10)
print('lines shape =', lines.shape)

edge = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)

for i in lines:
    pt1 = (i[0, 0], i[0, 1])
    pt2 = (i[0, 2], i[0, 3])
    cv2.line(edge, pt1, pt2, (0, 0, 255), 2)

cv2.imshow('img', img)
cv2.imshow('edge', edge)

cv2.waitKey()
cv2.destroyAllWindows()

lines shape = (68, 1, 4)


In [21]:
src = cv2.imread(Path(folder, 'plates.png'))
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

circles = cv2.HoughCircles(src_gray, cv2.HOUGH_GRADIENT, 1, 50,
                           param1=100, param2=50, minRadius=20, maxRadius=100)

for i in circles[0]:
    cx, cy, r = i
    cv2.circle(src, (int(cx), int(cy)), int(r), (0, 0 ,255), 2, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('src_gray', src_gray)

cv2.waitKey()
cv2.destroyAllWindows()

In [27]:
### labeling

src = cv2.imread(Path(folder, 'symbols.png'))
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(src_gray, 100, 255, cv2.THRESH_BINARY_INV)

cnts, labels, stats, centroids = cv2.connectedComponentsWithStats(mask)

for i in range(1, cnts):
    (x, y, w, h, area) = stats[i]

    if area > 50:
        cv2.rectangle(src, (x, y, w, h), (0, 0, 255), 2)
        cv2.putText(src, str(i), (x, y-4), 0, 1, (0, 0, 255), 1, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('src_gray', src_gray)
cv2.imshow('mask', mask)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
src = cv2.imread(Path(folder, 'symbols.png'))
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
_, mask = cv2.threshold(src_gray, 150, 255, cv2.THRESH_BINARY)
mask_open = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, iterations=2)

contours, hieracy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

for i in range(len(contours)):
    cv2.drawContours(src, contours, i, (0, 0, 255), 1, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('src_gray', src_gray)
cv2.imshow('mask', mask)

cv2.waitKey()
cv2.destroyAllWindows()

In [34]:
##

cap = cv2.VideoCapture(Path(folder, "PETS2000.avi"))

if not cap.isOpened():
    print('Video open failed')
    sys.exit()

while True:

    ret, frame = cap.read()

    if not ret:
        break

    cv2.imshow('frame', frame)

    if cv2.waitKey(20) == 27:
        break

cap.release() # 카메라 끄기
cv2.destroyAllWindows()

## 동영상 객체 검출

In [48]:
cap = cv2.VideoCapture(Path(folder, 'PETS2000.avi'))

if not cap.isOpened():
    print('video read failed')
    sys.exit()

ret, background = cap.read()
background_gray = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
background_gray = cv2.GaussianBlur(background_gray, (0, 0), 1)

while True:
    ret, frame = cap.read()

    if not ret:
        print('frame read failed')
        break
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.GaussianBlur(frame_gray, (0, 0), 1)

    diff_G = cv2.absdiff(frame_gray, background_gray)
    ret_g, mask_g = cv2.threshold(diff_G, 50, 255, cv2.THRESH_BINARY)

    cnts, labels, stats, centroids = cv2.connectedComponentsWithStats(mask_g)

    for i in range(1, cnts):
        x, y, w, h, area = stats[i]

        if area < 400:
            continue
        cv2.rectangle(frame, (x, y, w, h), (0, 0, 255), 2)

    cv2.imshow('frame', frame)
    cv2.imshow('diff_G', diff_G)
    cv2.imshow('mask_g', mask_g)

    if cv2.waitKey(20) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

frame read failed


### 이동 평균 배경차분

In [49]:
# accumulateWeighted(src, dst, alpha, mask) -> dst
# src: 입력영상
# dis: 출력영상 (32bit, 64bit)
# alpha : 축적가중치
# mask: 마스트 영상

In [58]:
cap = cv2.VideoCapture(Path(folder, 'PETS2000.avi'))

if not cap.isOpened():
    print('video read failed')
    sys.exit()

ret, back = cap.read()
back = cv2.cvtColor(back, cv2.COLOR_BGR2GRAY)
back = cv2.GaussianBlur(back, (0, 0), 1.) # np.uint8
fback = back.astype(np.float32)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.GaussianBlur(frame_gray, (0, 0), 1.)
    cv2.accumulateWeighted(frame_gray, fback, 0.005)
    back = fback.astype(np.uint8)
    diff = cv2.absdiff(frame_gray, back)
    ret, mask = cv2.threshold(diff, 50, 255, cv2.THRESH_BINARY)

    cnts, labels, stats, centroids = cv2.connectedComponentsWithStats(mask)

    for i in range(1, cnts):
        x, y, w, h, area = stats[i]
        if area > 200:
            cv2.rectangle(frame, (x, y, w, h), (0, 0, 255), 2)

    cv2.imshow('frame', frame)
    cv2.imshow('mask', mask)
    cv2.imshow('back', back)

    if cv2.waitKey(20) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('webcam open failed')
    sys.exit()

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"Frame width = {w}, height = {h}")

while True:
    ret, frame = cap.read()

    if not ret:
        break
    
    frame_Gau = cv2.GaussianBlur(frame, (0, 0), 1)
    edge = cv2.Canny(frame_Gau, 50, 200)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray_inv = 255 - frame_gray

    cv2.imshow('frame', frame)
    cv2.imshow('edge', edge)
    cv2.imshow('frame_gray', frame_gray)
    cv2.imshow('frame_gray_inv', frame_gray_inv)

    if cv2.waitKey(20) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()